# **APA Project**

**Read in data**

In [1]:
import pandas as pd
data = pd.read_csv('players_22.csv')
data.head(3)

<ipython-input-1-0abab749b0ce>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('players_22.csv')


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png


In [2]:
data.shape

(19239, 110)

# **Data Cleaning**

In [3]:
#Drop Goalkeepers
data.drop(data[data['player_positions'] == 'GK'].index, inplace=True)

In [4]:
#Remove columns with external links - no use in machine learning
data = data[data.columns.drop(list(data.filter(regex='url')))]

In [5]:
#Drop columns where target variable is not stored
data = data.dropna(subset=['value_eur'])

In [6]:
#Drop columns not needed: jerseys, names, goalkeeping statistics, club/nationality stats, id, real face in game
data = data.drop(['long_name', 'dob', 'nation_jersey_number', 'short_name', 'club_jersey_number', 'sofifa_id', 'real_face', 'gk',
 'goalkeeping_diving','goalkeeping_handling','goalkeeping_kicking','goalkeeping_positioning','goalkeeping_reflexes',
 'goalkeeping_speed', 'nationality_id', 'player_tags', 'player_traits', 'club_name', 'club_loaned_from', 'club_joined',
  'league_name', 'player_positions', 'nationality_name', 'body_type'  ], axis=1)
data.head()

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_position,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,93,93,78000000.0,320000.0,34,170,72,73.0,1.0,RW,...,66+3,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3
1,92,92,119500000.0,270000.0,32,185,81,21.0,1.0,ST,...,64+3,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3
2,91,91,45000000.0,270000.0,36,187,83,11.0,1.0,ST,...,63+3,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3
3,91,91,129000000.0,270000.0,29,175,68,73.0,1.0,LW,...,67+3,63+3,63+3,63+3,67+3,62+3,50+3,50+3,50+3,62+3
4,91,91,125500000.0,350000.0,30,181,70,10.0,1.0,RCM,...,79+3,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3


In [7]:
#Identify positional columns
data.columns[50:100]

Index(['mentality_composure', 'defending_marking_awareness',
       'defending_standing_tackle', 'defending_sliding_tackle', 'ls', 'st',
       'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm',
       'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb',
       'rcb', 'rb'],
      dtype='object')

In [8]:
#Drop positional columns
data = data.loc[:, :'defending_sliding_tackle']

In [9]:
# View unique dtypes of all variables
unique_dtypes = data.dtypes.unique()
unique_dtypes.tolist()

[dtype('int64'), dtype('float64'), dtype('O')]

In [10]:
#Identify non-numeric cols for encoding
sel_cols = list(data.select_dtypes(include='O'))
print(sel_cols)

['club_position', 'nation_position', 'preferred_foot', 'work_rate']


**Missing Data**

In [11]:
data.shape

(17041, 54)

In [12]:
data.isna().sum()

,0
overall,0
potential,0
value_eur,0
wage_eur,0
age,0
height_cm,0
weight_kg,0
club_team_id,0
league_level,0
club_position,0


In [13]:
data.isnull().sum().sum()

33889

In [14]:
#National team information: As data is missing for 16434/17041(96% of values) decision is made to drop this information
data.drop('nation_team_id', inplace=True, axis=1)

In [15]:
data.drop('nation_position', inplace=True, axis=1)

In [16]:
data.isna().sum()

,0
overall,0
potential,0
value_eur,0
wage_eur,0
age,0
height_cm,0
weight_kg,0
club_team_id,0
league_level,0
club_position,0


**Encoding Categorical Variables**

Encoding Preferred Foot

In [17]:
#Preferred Foot
data['preferred_foot'].unique()

array(['Left', 'Right'], dtype=object)

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['preferred_foot'] = le.fit_transform(data['preferred_foot'])

In [19]:
#Preferred Foot
data['preferred_foot'].unique()

array([0, 1])

Encoding Position

In [20]:
#Position Definitions
att = ['RW', 'RS', 'ST' , 'LW',  'CF', 'LS',  'LF',  'RF']
mid = ['RCM',  'CDM',  'RDM', 'LCM',  'LDM', 'CM','RM', 'CAM', 'LM',  'LAM', 'RAM']
defe = [ 'LCB', 'LWB', 'LB', 'RCB', 'RB', 'CB', 'RWB']

In [22]:
data['club_position'].unique()

array(['RW', 'ST', 'LW', 'RCM', 'CF', 'CDM', 'LCB', 'RDM', 'RS', 'LCM',
       'SUB', 'CAM', 'RCB', 'LDM', 'LB', 'RB', 'LM', 'RM', 'LS', 'CB',
       'RES', 'RWB', 'RF', 'CM', 'LWB', 'LAM', 'LF', 'RAM'], dtype=object)

In [24]:
#Put Positions into 3 categories - Club Position col 9
new_positions = []
index = 0
for row in data.iterrows():
  x = (data.iloc[index, 9])
  if x == 'RW' or x == 'RS' or x == 'ST' or x == 'LW' or  x == 'CF' or x == 'LS' or  x == 'LF' or  x == 'RF':
      new_positions.append('ATT')
  elif x == 'RCM' or  x == 'CDM' or  x == 'RDM' or x == 'LCM' or  x == 'LDM' or x == 'CM' or x == 'RM' or x == 'CAM' or x == 'LM' or  x == 'LAM' or x == 'RAM':
      new_positions.append('MID')
  elif x == 'LCB' or x == 'LWB' or x == 'LB' or x == 'RCB' or x == 'RB' or x == 'CB' or x == 'RWB':
      new_positions.append('DEF')
  else:
      new_positions.append('other')
  index = index + 1
new_club_positions = new_positions

In [25]:
data['club_position'] = new_club_positions

In [26]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['club_position'] = le.fit_transform(data['club_position'])#
data['club_position'].unique()

array([0, 2, 1, 3])

In [27]:
#Identify non-numeric cols for encoding
sel_cols = list(data.select_dtypes(include='O'))
print(sel_cols)

['work_rate']


Encoding Work Rate

In [28]:
data['work_rate'].unique()

array(['Medium/Low', 'High/Medium', 'High/Low', 'High/High',
       'Medium/High', 'Medium/Medium', 'Low/High', 'Low/Medium',
       'Low/Low'], dtype=object)

In [29]:
#Split into two columsn
data[['att_wr', 'def_wr']] = data['work_rate'].str.split('/', expand=True)
data['att_wr']

,att_wr
0,Medium
1,High
2,High
3,High
4,High
...,...
19234,Medium
19235,Medium
19236,Medium
19237,Medium


In [30]:
data['def_wr']

,def_wr
0,Low
1,Medium
2,Low
3,Medium
4,High
...,...
19234,Medium
19235,Medium
19236,Medium
19237,Medium


In [31]:
data['att_wr'].unique()

array(['Medium', 'High', 'Low'], dtype=object)

In [32]:
data['def_wr'].unique()

array(['Low', 'Medium', 'High'], dtype=object)

In [33]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['att_wr'] = le.fit_transform(data['att_wr'])

In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['def_wr'] = le.fit_transform(data['def_wr'])

In [35]:
#Drop non-numeric col
data.drop('work_rate', inplace=True, axis=1)

In [36]:
#Identify non-numeric cols for encoding
sel_cols = list(data.select_dtypes(include='O'))
print(sel_cols)

[]


In [39]:
# View unique dtypes of all variables
# Ensure all numeric
unique_dtypes = data.dtypes.unique()
unique_dtypes.tolist()

[dtype('int64'), dtype('float64')]

In [40]:
data.head()

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_position,...,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,att_wr,def_wr
0,93,93,78000000.0,320000.0,34,170,72,73.0,1.0,0,...,40,93,95,75,96,20,35,24,2,1
1,92,92,119500000.0,270000.0,32,185,81,21.0,1.0,0,...,49,95,81,90,88,35,42,19,0,2
2,91,91,45000000.0,270000.0,36,187,83,11.0,1.0,0,...,29,95,76,88,95,24,32,24,0,1
3,91,91,129000000.0,270000.0,29,175,68,73.0,1.0,0,...,37,86,90,93,93,35,32,29,0,2
4,91,91,125500000.0,350000.0,30,181,70,10.0,1.0,2,...,66,88,94,83,89,68,65,53,0,0


In [41]:
data.columns[data.isna().any()].tolist()

['release_clause_eur']

Drop all rows with target variable as missing.

In [42]:
data.dropna(subset=['release_clause_eur'],inplace=True)

In [43]:
print(data.isnull().values.any())

False


In [44]:
data.columns[data.isna().any()].tolist()

[]

In [45]:
data.isnull().sum().sum()

0

In [46]:
data

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_position,...,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,att_wr,def_wr
0,93,93,78000000.0,320000.0,34,170,72,73.0,1.0,0,...,40,93,95,75,96,20,35,24,2,1
1,92,92,119500000.0,270000.0,32,185,81,21.0,1.0,0,...,49,95,81,90,88,35,42,19,0,2
2,91,91,45000000.0,270000.0,36,187,83,11.0,1.0,0,...,29,95,76,88,95,24,32,24,0,1
3,91,91,129000000.0,270000.0,29,175,68,73.0,1.0,0,...,37,86,90,93,93,35,32,29,0,2
4,91,91,125500000.0,350000.0,30,181,70,10.0,1.0,2,...,66,88,94,83,89,68,65,53,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,52,70000.0,1000.0,22,180,64,112541.0,1.0,3,...,42,38,43,42,37,38,43,48,2,2
19235,47,59,110000.0,500.0,19,175,70,445.0,1.0,3,...,39,51,49,44,47,37,44,47,2,2
19236,47,55,100000.0,500.0,21,178,72,111131.0,1.0,3,...,38,45,46,39,36,38,44,48,2,2
19237,47,60,110000.0,500.0,19,173,66,111131.0,1.0,3,...,14,47,40,49,47,10,14,11,2,2


In [47]:
from google.colab import files
data.to_csv('final_clean_data.csv', index=None)
files.download("final_clean_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>